<a href="https://colab.research.google.com/github/isaacsgabe/School/blob/main/NLP/sentiment_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework #3 is to study Sentiment Analysis with five types of models:

1.	Rule-based
2.	Bag of Words
3.	Shallow embedding with CNN
4.  LSTM
5.	Transformer Models

I got this idea from my student Itay Etelis and his Huggingface depo is at:
https://huggingface.co/pig4431

We will discuss all five possibilities in class.

## YELP Reviews DataSet
The **YELP** reviews dataset consists of reviews from Yelp. It is extracted from the Yelp Dataset Challenge 2015 data.

The Yelp reviews full star dataset is constructed by Xiang Zhang (xiang.zhang@nyu.edu) from the Yelp Dataset Challenge 2015. It is first used as a text classification benchmark in the following paper: Xiang Zhang, Junbo Zhao, Yann LeCun. Character-level Convolutional Networks for Text Classification. Advances in Neural Information Processing Systems 28 (NIPS 2015).  Reviews with 1, 2 stars has been marked negative.
Reviews with 4, 5 stars has been marked positive.

Reviews with 3 stars has been filtered.

Full information about this dataset is at:
https://www.kaggle.com/datasets/ilhamfp31/yelp-review-dataset

You can either download this dataset from there, or use Itay's code with a train/test/validation partition:

In [ ]:
!pip install datasets -q

import datasets
datasets.logging.set_verbosity_error()
datasets.disable_progress_bar()

from datasets import load_dataset
yelp = load_dataset("pig4431/yelp_train25k_test5k_valid5k")
yelp_train = yelp['train']
yelp_validate = yelp['validate']
yelp_test = yelp['test']

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
yelp_train[3]

{'label': 1,
 'text': "Best hot dog i've ever had!  This place was absolutely great.  Such an interesting combination of flavors, but it definitely works."}

The advantage to having a testing dataset is that you can tune certain parameters and then see if they validate correctly.  This can be done for all 5 models, but is particularly easy for the first two models: Rule-based and Bag-of-Words models.

For example, you can use the rule based approach, VADER:

https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/
by installing:


In [ ]:
!pip install vaderSentiment


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj = SentimentIntensityAnalyzer()
sentiment_dict = sid_obj.polarity_scores("I love programming!")

In [ ]:
print(sentiment_dict)

{'neg': 0.0, 'neu': 0.308, 'pos': 0.692, 'compound': 0.6696}


Note that this model yields four different scores: neg, neu, pos, and compound. We'll talk about these four possibilities in class.

Note that the compound score is meant to be based of *all* lexicon ratings and is normalized between -1 (most extreme negative) and +1 (most extreme positive).  In the link above they suggest the thresholds for this value:

positive sentiment : (compound score >= 0.05)
neutral sentiment : (compound score > -0.05) and (compound score < 0.05)
negative sentiment : (compound score <= -0.05)

However, please check if this is check the actually best threshold for the given dataset but checking a range of values such as:

for compound_score in np.arange(-1, 1, 0.1):

which can be checked in test dataset and then validated.

Please do so!

Please implement the rule-based model here with the hyperparameter tuning for the compound_score.

In [ ]:
def predict_sentiment(text, threshold):
    # Calculate sentiment scores for the text
    scores = sid_obj.polarity_scores(text)
    compound_score = scores['compound']

    # Apply threshold to predict sentiment
    if compound_score >= threshold:
        return 'positive'
    else:
        return 'negative'

In [ ]:
import numpy as np

best_threshold = None
best_accuracy = 0

# Define the range of compound score thresholds to test
thresholds_to_test = np.arange(-1, 1.1, 0.1)

for threshold in thresholds_to_test:
    correct_predictions = 0
    total_predictions = len(yelp_validate)

    for text, label in zip(yelp_validate['text'], yelp_validate['label']):
        rounded_threshold = round(threshold, 1)
        predicted_sentiment = predict_sentiment(text, rounded_threshold)
        pred = 0
        if predicted_sentiment == 'positive':
          pred = 1
        else:
          pred = -1
        if pred == label:
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    print(f"Threshold: {rounded_threshold}, Accuracy: {accuracy}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = rounded_threshold

print(f"Best threshold: {best_threshold}, Best accuracy: {best_accuracy}")

# Apply the best threshold on the test set
test_correct_predictions = 0
test_total_predictions = len(yelp_test)

for text, label in zip(yelp_test['text'], yelp_test['label']):
    predicted_sentiment = predict_sentiment(text, best_threshold)
    pred = 0
    if predicted_sentiment == 'positive':
      pred = 1
    else:
      pred = -1
    if pred == label:
        test_correct_predictions += 1

test_accuracy = test_correct_predictions / test_total_predictions
print(f"Test Accuracy with best threshold: {test_accuracy}")


Threshold: -1.0, Accuracy: 0.5126
Threshold: -0.9, Accuracy: 0.559
Threshold: -0.8, Accuracy: 0.5944
Threshold: -0.7, Accuracy: 0.6192
Threshold: -0.6, Accuracy: 0.6388
Threshold: -0.5, Accuracy: 0.6594
Threshold: -0.4, Accuracy: 0.6748
Threshold: -0.3, Accuracy: 0.6852
Threshold: -0.2, Accuracy: 0.6964
Threshold: -0.1, Accuracy: 0.707
Threshold: -0.0, Accuracy: 0.7136
Threshold: 0.1, Accuracy: 0.721
Threshold: 0.2, Accuracy: 0.7302
Threshold: 0.3, Accuracy: 0.7388
Threshold: 0.4, Accuracy: 0.7464
Threshold: 0.5, Accuracy: 0.7552
Threshold: 0.6, Accuracy: 0.7684
Threshold: 0.7, Accuracy: 0.768
Threshold: 0.8, Accuracy: 0.7692
Threshold: 0.9, Accuracy: 0.7438
Threshold: 1.0, Accuracy: 0.4874
Best threshold: 0.8, Best accuracy: 0.7692
Test Accuracy with best threshold: 0.7686


Now, please implement a Bag of Words model.  Check if feature selection works and validate on the validation dataset.

Which words are most strongly correlated to positive sentiment?  Which are strongly correlated to negative sentiment?  One way to check is find those words with high PMI to positive words (e.g. excellent, great) and those with negative words (bad, terrible).

Please work similarly to what we did in the first homework and feel free to adapt your PMI corde from there.

Also, similar to the first homework, using the train and test datasets, find the number of features to choose.  Then validate this amount using the validation dataset.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np

X_train = yelp_train['text']
y_train = yelp_train['label']
X_validate = yelp_validate['text']
y_validate = yelp_validate['label']

# Vectorize the text using Bag of Words model
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_validate_bow = vectorizer.transform(X_validate)

# Feature selection (select top-N features)
# You can adjust the value of k as needed
k = 1000  # Number of top features to select
selector = SelectKBest(chi2, k=k)
X_train_selected = selector.fit_transform(X_train_bow, y_train)
X_validate_selected = selector.transform(X_validate_bow)

# Train a classifier (Multinomial Naive Bayes)
clf = MultinomialNB()
clf.fit(X_train_selected, y_train)

# Evaluate the classifier on the validation set
y_pred = clf.predict(X_validate_selected)
accuracy = accuracy_score(y_validate, y_pred)
print("Validation Accuracy:", accuracy)

# Get the top-N features (words) most strongly correlated to positive and negative sentiment
feature_names = np.array(vectorizer.get_feature_names_out())
selected_feature_names = feature_names[selector.get_support()]

# Get the coefficients (log probabilities) of the features
coefficients = clf.feature_log_prob_

# Sort the coefficients and feature names by coefficient values
sorted_indices = np.argsort(coefficients[1] - coefficients[0])  # for binary classification, we only have two classes

# Top N positive features
top_positive_indices = sorted_indices[-10:][::-1]
top_positive_words = selected_feature_names[top_positive_indices]

# Top N negative features
top_negative_indices = sorted_indices[:10]
top_negative_words = selected_feature_names[top_negative_indices]

print("Top 5 Positive Words:", top_positive_words)
print("Top 5 Negative Words:", top_negative_words)

Validation Accuracy: 0.8718
Top 5 Positive Words: ['deliciously' 'divine' 'biz_photos' 'halo' 'downside' 'pleasantly'
 'wonderfully' 'exceeded' 'ngreat' 'gem']
Top 5 Negative Words: ['ow' 'unprofessional' 'inedible' 'rudest' 'disgusted' 'worst' 'rudely'
 'disgusting' 'pathetic' 'questioned']


Now, please implement an embedded model with a CNN similar to the previous homework. Feel free to use either Glove *or* Word2Vec -- whichever worked better for you in the previous homework. No need to check both.

In [ ]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras.models import Sequential
import gensim.downloader as api
from keras.layers import TextVectorization
import tensorflow as tf


glove_model = api.load("glove-wiki-gigaword-300")
embedding_dim = glove_model.vector_size


In [ ]:
X_train = yelp_train['text']
y_train = yelp_train['label']
X_test = yelp_validate['text']
y_test = yelp_validate['label']


In [ ]:
y_test[1]

1

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Define a function to remove stopwords
def remove_stopwords(text_list):
    filtered_text_list = []
    for text in text_list:
        # Convert to string if the element is not already a string
        if not isinstance(text, str):
            text = str(text)
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in ENGLISH_STOP_WORDS]
        filtered_text = ' '.join(filtered_words)
        filtered_text_list.append(filtered_text)
    return filtered_text_list

# Apply the remove_stopwords function to the list of texts
X_train = remove_stopwords(X_train)
y_train = remove_stopwords(y_train)


In [ ]:
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(X_train).batch(128)
vectorizer.adapt(text_ds)

X_tra = vectorizer(np.array([[s] for s in X_train])).numpy()
X_te = vectorizer(np.array([[s] for s in X_test])).numpy()

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
vocab_size = len(word_index)
num_tokens = len(voc) + 2
embedding_matrix = np.zeros((num_tokens, embedding_dim))

hits = 0
misses = 0

# Prepare embedding matrix
for word, i in word_index.items():
    if word in glove_model:
      hits += 1
      embedding_matrix[i] = glove_model[word]
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17588 words (2412 misses)


In [ ]:
from keras.layers import Embedding
import keras

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
from keras import layers

def createModel():
  int_sequences_input = keras.Input(shape=(None,), dtype="int64")
  embedded_sequences = embedding_layer(int_sequences_input)
  x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
  x = layers.MaxPooling1D(5)(x)
  x = layers.Conv1D(128, 5, activation="relu")(x)
  x = layers.MaxPooling1D(5)(x)
  x = layers.Conv1D(128, 5, activation="relu")(x)
  x = layers.GlobalMaxPooling1D()(x)
  x = layers.Dense(128, activation="relu")(x)
  x = layers.Dropout(0.5)(x)
  preds = layers.Dense(2, activation="sigmoid")(x)
  model = keras.Model(int_sequences_input, preds)
  model.summary()
  return model

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
model = createModel()
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"],
)
type(y_train)
model.fit(X_tra, y_train_encoded, batch_size=128, epochs=10, validation_data=(X_te, y_test_encoded))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 300)         6000600   
                                                                 
 conv1d (Conv1D)             (None, None, 128)         192128    
                                                                 
 max_pooling1d (MaxPooling1  (None, None, 128)         0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, None, 128)         0         
 g1D)                                                        

Next, please try a LSTM model with Keras' word embedding.
I personally liked the tutorial here:

https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/.  Note that certain lines like:

top_words = 5000

(X_train, y_train), (X_test, y_test) = imdb.load_data

(num_words=top_words)

will need to be tweaked. Feel free to use the number of top_words you had in the Bag-of_words model for the parameter top_words.

**LSTM:** https://colab.research.google.com/drive/1AZ7YpNaknv9AI0Z_nJHzw4Z2qIDxJkGC?usp=sharing

In [ ]:
LSTM: https://colab.research.google.com/drive/1AZ7YpNaknv9AI0Z_nJHzw4Z2qIDxJkGC?usp=sharing

Now try a transformer model.  While you can train it from scratch, I suggest you don't and use something what we discussed in class:
https://colab.research.google.com/drive/15fisDt6RHTdFnkskokD9-jJ9luEbv-z3?usp=sharing

While this dataset was developed for SST2 (Stanford Sentiment Treebank v2), feel free to use it "as is" and without any fine tuning to the model. However, do please check if a different sentiment threshold would work better for this specific datset similar to what you did in the Vader model. It may be that the threshold will need to be tuned here too.

**Transformer: https://colab.research.google.com/drive/1IZ0OHp8QCKQg5Ua1e6SyGtEu8r-tByTX?usp=sharing**

In [ ]:
Transformer: https://colab.research.google.com/drive/1IZ0OHp8QCKQg5Ua1e6SyGtEu8r-tByTX?usp=sharing

Make sure to reflect about these models and the differences in their performance.